In [9]:
import numpy as np, pandas as pd
from matplotlib.pyplot import subplots
from sklearn.linear_model import \
     (LinearRegression,
      LogisticRegression,
      Lasso)
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from ISLP import load_data
from ISLP.models import ModelSpec as MS
from sklearn.model_selection import (train_test_split,
      GridSearchCV)

import torch
from torch import nn
from torch.optim import RMSprop
from torch.utils.data import TensorDataset

from torchmetrics import (MeanAbsoluteError,
                          R2Score)
from torchinfo import summary

from pytorch_lightning import Trainer
from pytorch_lightning.loggers import CSVLogger

from pytorch_lightning import seed_everything
seed_everything(0, workers=True)
torch.use_deterministic_algorithms(True, warn_only=True)

from ISLP.torch import (SimpleDataModule,
                        SimpleModule,
                        ErrorTracker,
                        rec_num_workers)

Seed set to 0


In [10]:
NYSE = load_data('NYSE')
print(NYSE.head())
cols = ['DJ_return', 'log_volume', 'log_volatility']
print(NYSE.columns)

           day_of_week  DJ_return  log_volume  log_volatility  train
date                                                                
1962-12-03         mon  -0.004461    0.032573      -13.127403   True
1962-12-04        tues   0.007813    0.346202      -11.749305   True
1962-12-05         wed   0.003845    0.525306      -11.665609   True
1962-12-06        thur  -0.003462    0.210182      -11.626772   True
1962-12-07         fri   0.000568    0.044187      -11.728130   True
Index(['day_of_week', 'DJ_return', 'log_volume', 'log_volatility', 'train'], dtype='object')


In [84]:
# Standardize the features (z-score normalization)
X = pd.DataFrame(StandardScaler(
                     with_mean=True,
                     with_std=True).fit_transform(NYSE[cols]),
                 columns=NYSE[cols].columns,
                 index=NYSE.index)

# Create a lagged data
for lag in range(1, 6):
    for col in cols:
        newcol = np.zeros(X.shape[0]) * np.nan
        newcol[lag:] = X[col].values[:-lag]
        X.insert(len(X.columns), "{0}_{1}".format(col, lag), newcol)
X.insert(len(X.columns), 'train', NYSE['train'])
X = X.dropna()

# extract the response and features
Y, train = X['log_volume'], X['train']
X = X.drop(columns=['train'] + cols)

print(X.columns)

Index(['DJ_return_1', 'log_volume_1', 'log_volatility_1', 'DJ_return_2',
       'log_volume_2', 'log_volatility_2', 'DJ_return_3', 'log_volume_3',
       'log_volatility_3', 'DJ_return_4', 'log_volume_4', 'log_volatility_4',
       'DJ_return_5', 'log_volume_5', 'log_volatility_5'],
      dtype='object')


## Prepare a lagged data set

In [24]:
# reverse the the order of columns to have lag 5 first, then lag 4, ..., lag 1
ordered_cols = []
for lag in range(5,0,-1):
    for col in cols:
        ordered_cols.append('{0}_{1}'.format(col, lag))
X = X.reindex(columns=ordered_cols)
print(X.shape)
X.columns


(6046, 15)


Index(['DJ_return_5', 'log_volume_5', 'log_volatility_5', 'DJ_return_4',
       'log_volume_4', 'log_volatility_4', 'DJ_return_3', 'log_volume_3',
       'log_volatility_3', 'DJ_return_2', 'log_volume_2', 'log_volatility_2',
       'DJ_return_1', 'log_volume_1', 'log_volatility_1'],
      dtype='object')

## First, we fit the above data set with an AR using a Linear Regression Model

In [25]:
M = LinearRegression()
M.fit(X[train], Y[train])
M.score(X[~train], Y[~train])

0.4128912938562521

## 2nd: reshape the data for RNN model and fit an RNN model

In [26]:
# reshape the data for RNN model
# an RNN require the input in the shape of (batch_size, sequence_length, num_features)
# -1: infers the batch size automatically
# 5: sequence length (number of time steps)
# 3: number of features at each time step
X_rnn = X.to_numpy().reshape((-1,5,3))
print(X_rnn.shape)
X_rnn.shape[1]

# Prepare the data set for pytorch
datasets = []
for mask in [train, ~ train]:
    X_rnn_t = torch.tensor(X_rnn[mask].astype(np.float32))
    Y_t = torch.tensor(Y[mask].astype(np.float32))
    datasets.append(TensorDataset(X_rnn_t, Y_t))

nyse_train, nyse_test = datasets
print(nyse_train.tensors[0].shape)
print(nyse_train.tensors[1].shape)

(6046, 5, 3)
torch.Size([4276, 5, 3])
torch.Size([4276])
torch.Size([4276, 5, 3])
torch.Size([4276])


/tmp/ipykernel_41284/3508433103.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  Y_t = torch.tensor(Y[mask].astype(np.float32))
/tmp/ipykernel_41284/3508433103.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  Y_t = torch.tensor(Y[mask].astype(np.float32))


In [43]:
class NYSEModel(nn.Module): # inherit from nn.Module
    def __init__(self):
        super(NYSEModel, self).__init__() # initialize the parent class
        self.rnn = nn.RNN(3, # input size (num_features)
                          12, # hidden size
                          batch_first=True)
        self.dense = nn.Linear(12, 1)
        self.dropout = nn.Dropout(0.1) # 10% dropout
    
    def forward(self, x):
        val, h_n = self.rnn(x)
        val = self.dense(self.dropout(val[:,-1]))
        return torch.flatten(val)

nyse_model = NYSEModel()

In [44]:
summary(nyse_model,
        input_data=X_rnn_t,
        col_names=['input_size',
                   'output_size',
                   'num_params'])
# The RNN has:
# RNN Layer Parameters:
# An RNN has two sets of weights and biases:

# Input-to-Hidden weights: W_ih

# Shape: (hidden_size × input_size) = 12 × 3 = 36 parameters
# Hidden-to-Hidden weights: W_hh

# Shape: (hidden_size × hidden_size) = 12 × 12 = 144 parameters
# Input-to-Hidden bias: b_ih

# Shape: (hidden_size) = 12 = 12 parameters
# Hidden-to-Hidden bias: b_hh

# Shape: (hidden_size) = 12 = 12 parameters
# Total RNN parameters: 36 + 144 + 12 + 12 = 204

# the dense layer has: (12 + 1) = 13 parameters

# => Total parameters: 204 + 13 = 217 parameters

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #
NYSEModel                                [1770, 5, 3]              [1770]                    --
├─RNN: 1-1                               [1770, 5, 3]              [1770, 5, 12]             204
├─Dropout: 1-2                           [1770, 12]                [1770, 12]                --
├─Linear: 1-3                            [1770, 12]                [1770, 1]                 13
Total params: 217
Trainable params: 217
Non-trainable params: 0
Total mult-adds (M): 1.83
Input size (MB): 0.11
Forward/backward pass size (MB): 0.86
Params size (MB): 0.00
Estimated Total Size (MB): 0.97

In [ ]:
max_num_workers = rec_num_workers()
print("max_num_workers:", max_num_workers)

nyse_dm = SimpleDataModule(nyse_train,
                           nyse_test,
                           num_workers=min(4, max_num_workers),
                           validation=nyse_test,
                           batch_size=64)
# validate the train_dataloader
for idx, (x, y) in enumerate(nyse_dm.train_dataloader()):
    out = nyse_model(x)
    print(y.size(), out.size()) # y.size() should be equal to out.size()
    if idx >= 2:
        break

max_num_workers: 16
torch.Size([64]) torch.Size([64])
torch.Size([64]) torch.Size([64])
torch.Size([64]) torch.Size([64])


In [51]:
# nyse_model.parameters() => list of model parameters, 217 params
# lr = 0.001 => learning rate
nyse_optimizer = RMSprop(nyse_model.parameters(),
                         lr=0.001)
nyse_module = SimpleModule.regression(nyse_model,
                                      optimizer=nyse_optimizer,
                                      metrics={'r2':R2Score()})

In [52]:
nyse_trainer = Trainer(deterministic=True,
                       max_epochs=200,
                       callbacks=[ErrorTracker()])# init a trainer
nyse_trainer.fit(nyse_module,
                 datamodule=nyse_dm) # start the training here
nyse_trainer.test(nyse_module,
                  datamodule=nyse_dm)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type      | Params | Mode 
--------------------------------------------
0 | model | NYSEModel | 217    | train
1 | loss  | MSELoss   | 0      | train
--------------------------------------------
217       Trainable params
0         Non-trainable params
217       Total params
0.001     Total estimated model params size (MB)
5         Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=200` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss           0.6153033375740051
         test_r2            0.4160470962524414
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.6153033375740051, 'test_r2': 0.4160470962524414}]

# (b) fit a nonlinear AR model

In [87]:
# Prepare the data 
X_nl = X.to_numpy()

#Prepare a datamodule
# Prepare the data set for pytorch
datasets = []
for mask in [train, ~ train]:
    X_nl_masked = torch.tensor(X_nl[mask].astype(np.float32))
    Y_nl_masked = torch.tensor(Y[mask].astype(np.float32))
    datasets.append(TensorDataset(X_nl_masked, Y_nl_masked))

nyse_nl_train, nyse_nl_test = datasets
print(nyse_nl_train.tensors[0].shape)
print(nyse_nl_train.tensors[1].shape)
# Create a data module for training
nyse_nl_dm = SimpleDataModule(nyse_nl_train,
                           nyse_nl_test,
                           num_workers=4,
                           validation=nyse_nl_test,
                           batch_size=64)

torch.Size([4276, 15])
torch.Size([4276])


/tmp/ipykernel_41284/2209135852.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  Y_nl_masked = torch.tensor(Y[mask].astype(np.float32))
/tmp/ipykernel_41284/2209135852.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  Y_nl_masked = torch.tensor(Y[mask].astype(np.float32))


Build a nonlinear AR Model

In [88]:
class NonLinearARModel(nn.Module):

    def __init__(self, input_size):
        super(NonLinearARModel, self).__init__()
        self.flatten = nn.Flatten()
        self.sequential = nn.Sequential(
            nn.Linear(input_size, 32), # input to hidden, use 32 hidden units
            nn.ReLU(), # activation function, inject non-linearity
            nn.Dropout(0.1), # 10% dropout
            nn.Linear(32, 1)) # hidden to output

    def forward(self, x):
        x = self.flatten(x)
        return torch.flatten(self.sequential(x))
    
nl_model = NonLinearARModel(X.shape[1])
print(X.shape)

# Convert X to tensor for summary
X_nl_torch = torch.tensor(X.to_numpy().astype(np.float32))

# Show model summary
summary(nl_model,
        input_data=X_nl_torch,
        col_names=['input_size',
                   'output_size',
                   'num_params'])

(6046, 15)


Layer (type:depth-idx)                   Input Shape               Output Shape              Param #
NonLinearARModel                         [6046, 15]                [6046]                    --
├─Flatten: 1-1                           [6046, 15]                [6046, 15]                --
├─Sequential: 1-2                        [6046, 15]                [6046, 1]                 --
│    └─Linear: 2-1                       [6046, 15]                [6046, 32]                512
│    └─ReLU: 2-2                         [6046, 32]                [6046, 32]                --
│    └─Dropout: 2-3                      [6046, 32]                [6046, 32]                --
│    └─Linear: 2-4                       [6046, 32]                [6046, 1]                 33
Total params: 545
Trainable params: 545
Non-trainable params: 0
Total mult-adds (M): 3.30
Input size (MB): 0.36
Forward/backward pass size (MB): 1.60
Params size (MB): 0.00
Estimated Total Size (MB): 1.96

Init model and optimizer and start the training

In [89]:

nl_optimizer = RMSprop(nl_model.parameters(),
                           lr=0.001)
nl_module = SimpleModule.regression(nl_model,
                                        optimizer=nl_optimizer,
                                        metrics={'r2':R2Score()}) # wrapper for torch Litening module

# create a trainer
nl_trainer = Trainer(deterministic=True,
                         max_epochs=200,
                         callbacks=[ErrorTracker()])

# fit the nonlinear AR model to the data
nl_trainer.fit(nl_module, datamodule=nyse_nl_dm)
nl_trainer.test(nl_module, datamodule=nyse_nl_dm) 

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type             | Params | Mode 
---------------------------------------------------
0 | model | NonLinearARModel | 545    | train
1 | loss  | MSELoss          | 0      | train
---------------------------------------------------
545       Trainable params
0         Non-trainable params
545       Total params
0.002     Total estimated model params size (MB)
8         Modules in train mode
0         Modules in eval mode

  | Name  | Type             | Params | Mo

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=200` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss           0.6119141578674316
         test_r2            0.41926372051239014
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.6119141578674316, 'test_r2': 0.41926372051239014}]

# Add day_of_week and month features

## fit RNN model

In [69]:
# Process the data to add day_of_week and month feature

# Reformularize X again
# Standardize the features (z-score normalization)
X = pd.DataFrame(StandardScaler(
                     with_mean=True,
                     with_std=True).fit_transform(NYSE[cols]),
                 columns=NYSE[cols].columns,
                 index=NYSE.index)
# extract the response and features
Y, train = X['log_volume'], NYSE['train']
print(X.head())
X_day = X.copy()

# Add day_of_week features
X_day = pd.merge(X, 
                 pd.get_dummies(NYSE['day_of_week']),
                 left_index=True,
                 right_index=True) # add features for day of the week: mon, tues, wed, thu, fri
                                   # encode to dummies (categorical to binary features)

# Convert index to DatetimeIndex
X_day['date'] = pd.to_datetime(X_day.index)
X_day['month'] = X_day['date'].dt.month

X_day = pd.merge(X_day,
                 pd.get_dummies(X_day['month'], prefix='month'),
                 left_index=True,
                 right_index=True)
# remove date and month columns
X_day = X_day.drop(columns=['date', 'month'])

# drop first 5 row of the train
train = train.iloc[5:]

# Process data for RNN model
lag_size = 5 # lag 5 days
num_features = X_day.shape[1]
num_samples = X_day.shape[0]

X_rnn_extend_date = []
Y_rnn_extend_date = []
# We start looping from the 5th index to the end
# store a batch of 5-day data to feature X_rnn_extend_date
for idx in range(lag_size, num_samples):
    sample_x = X_day.iloc[idx-lag_size:idx].to_numpy()
    sample_y = Y.iloc[idx]
    X_rnn_extend_date.append(sample_x) # store the feature
    Y_rnn_extend_date.append(sample_y) # store the response

X_rnn_extend_date = np.array(X_rnn_extend_date)
Y_rnn_extend_date = np.array(Y_rnn_extend_date)
print(X_rnn_extend_date.shape) # check the shape
print(X_rnn_extend_date[0,:])
print(Y_rnn_extend_date.shape)

# Prepare the data set for pytorch
datasets_rnn_day_month = []
for mask in [train, ~ train]:
    X_rnn_t = torch.tensor(X_rnn_extend_date[mask].astype(np.float32))
    Y_t = torch.tensor(Y_rnn_extend_date[mask].astype(np.float32))
    datasets_rnn_day_month.append(TensorDataset(X_rnn_t, Y_t))

rnn_train, rnn_test = datasets_rnn_day_month
print(rnn_train.tensors[0].shape) # feature shape should be [4276,5,20]
print(rnn_train.tensors[1].shape) # response shape should be [4276]


            DJ_return  log_volume  log_volatility
date                                             
1962-12-03  -0.549823    0.175075       -4.357078
1962-12-04   0.905200    1.517291       -2.529058
1962-12-05   0.434813    2.283789       -2.418037
1962-12-06  -0.431397    0.935176       -2.366521
1962-12-07   0.046340    0.224779       -2.500970
(6046, 5, 20)
[[-0.549823342766741 0.17507496702556102 -4.357077862202008 True False
  False False False False False False False False False False False False
  False False True]
 [0.9051999460160775 1.5172907075939488 -2.5290576496438395 False True
  False False False False False False False False False False False False
  False False True]
 [0.43481275277098835 2.283789366833881 -2.4180369366103514 False False
  True False False False False False False False False False False False
  False False True]
 [-0.43139672833835296 0.9351755810204748 -2.366520943622522 False False
  False True False False False False False False False False False F

In [48]:
# Create RNN model with extended features (3 lagged + 5 days + 12 months = 20 features per timestep)
class NYSEModelExtended(nn.Module):
    def __init__(self):
        super(NYSEModelExtended, self).__init__()
        self.rnn = nn.RNN(num_features,  # input size: 3 lagged + 5 days + 12 months
                          24,  # hidden size
                          batch_first=True)
        self.dense = nn.Linear(24, 1)
        self.dropout = nn.Dropout(0.1)
    
    def forward(self, x):
        val, h_n = self.rnn(x)
        val = self.dense(self.dropout(val[:, -1]))
        return torch.flatten(val)

nyse_model_ext = NYSEModelExtended()

# Show model summary
summary(nyse_model_ext,
        input_data=X_rnn_t,  # Use first 10 samples
        col_names=['input_size', 'output_size', 'num_params'])

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #
NYSEModelExtended                        [1770, 5, 20]             [1770]                    --
├─RNN: 1-1                               [1770, 5, 20]             [1770, 5, 24]             1,104
├─Dropout: 1-2                           [1770, 24]                [1770, 24]                --
├─Linear: 1-3                            [1770, 24]                [1770, 1]                 25
Total params: 1,129
Trainable params: 1,129
Non-trainable params: 0
Total mult-adds (M): 9.81
Input size (MB): 0.71
Forward/backward pass size (MB): 1.71
Params size (MB): 0.00
Estimated Total Size (MB): 2.43

In [49]:
# Create DataModule for extended RNN model
nyse_dm_ext = SimpleDataModule(rnn_train,
                               rnn_test,
                               num_workers=4,
                               validation=rnn_test,
                               batch_size=64)

# Create optimizer and module
nyse_optimizer_ext = RMSprop(nyse_model_ext.parameters(), lr=0.001)
nyse_module_ext = SimpleModule.regression(nyse_model_ext,
                                          optimizer=nyse_optimizer_ext,
                                          metrics={'r2': R2Score()})

# Train the extended RNN model
nyse_trainer_ext = Trainer(deterministic=True,
                           max_epochs=200,
                           callbacks=[ErrorTracker()])

nyse_trainer_ext.fit(nyse_module_ext, datamodule=nyse_dm_ext)
nyse_trainer_ext.test(nyse_module_ext, datamodule=nyse_dm_ext)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type              | Params | Mode 
----------------------------------------------------
0 | model | NYSEModelExtended | 1.1 K  | train
1 | loss  | MSELoss           | 0      | train
----------------------------------------------------
1.1 K     Trainable params
0         Non-trainable params
1.1 K     Total params
0.005     Total estimated model params size (MB)
5         Modules in train mode
0         Modules in eval mode

  | Name  | Type              | Param

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=200` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss           0.6297796368598938
         test_r2            0.40230846405029297
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.6297796368598938, 'test_r2': 0.40230846405029297}]

## Fit AR Linear Regression Model with day_of_week and month features

In [80]:
X_ln_extend_date = []
Y_ln_extend_date = []
# We start looping from the 5th index to the end
# store a batch of 5-day data to feature X_rnn_extend_date
for idx in range(lag_size, num_samples):
    sample_x = X_day.iloc[idx-lag_size:idx].to_numpy()
    sample_x = sample_x.reshape(1, -1)  # Reshape from (5, 20) to (1, 100)
    sample_y = Y.iloc[idx]
    X_ln_extend_date.append(sample_x) # store the feature
    Y_ln_extend_date.append(sample_y) # store the response
X_ln_extend_date = np.array(X_ln_extend_date)
X_ln_extend_date = X_ln_extend_date.squeeze(1)  # Remove extra dimension: (n, 1, 100) -> (n, 100)
Y_ln_extend_date =  np.array(Y_ln_extend_date)
print(X_ln_extend_date.shape)
print(Y_ln_extend_date.shape)

(6046, 100)
(6046,)


In [71]:
# Fit AR Linear Regression model with extended features (lagged + day_of_week + month)
M_extended = LinearRegression()
M_extended.fit(X_ln_extend_date[train], Y_ln_extend_date[train])
M_extended.score(X_ln_extend_date[~train], Y_ln_extend_date[~train])

0.4650493457268383

## Fit Nonlinear AR Model with day_of_week and month features

In [73]:
# Prepare datasets for nonlinear AR model with extended features
datasets_nl_extended = []
for mask in [train, ~train]:
    X_nl_masked = torch.tensor(X_ln_extend_date[mask].astype(np.float32))
    Y_nl_masked = torch.tensor(Y_ln_extend_date[mask].astype(np.float32))
    datasets_nl_extended.append(TensorDataset(X_nl_masked, Y_nl_masked))

nyse_nl_train_ext, nyse_nl_test_ext = datasets_nl_extended

# Create a data module for training
nyse_nl_dm_ext = SimpleDataModule(nyse_nl_train_ext,
                                   nyse_nl_test_ext,
                                   num_workers=4,
                                   validation=nyse_nl_test_ext,
                                   batch_size=64)

print(f"Training data shape: {nyse_nl_train_ext.tensors[0].shape}")
print(f"Number of features: {X_ln_extend_date.shape[1]}")

Training data shape: torch.Size([4276, 100])
Number of features: 100


In [82]:
class NonLinearARModel(nn.Module):

    def __init__(self, input_size):
        super(NonLinearARModel, self).__init__()
        self.flatten = nn.Flatten()
        self.sequential = nn.Sequential(
            nn.Linear(input_size, 50), # input to hidden, use 32 hidden units
            nn.ReLU(), # activation function, inject non-linearity
            nn.Dropout(0.1), # 10% dropout
            nn.Linear(50, 1)) # hidden to output

    def forward(self, x):
        x = self.flatten(x)
        return torch.flatten(self.sequential(x))
    
# Create NonLinear AR Model with extended features (100 inputs: 5 lags × 20 features)
nl_model_ext = NonLinearARModel(X_ln_extend_date.shape[1])

# Convert X to tensor for summary
X_day_torch = torch.tensor(X_ln_extend_date.astype(np.float32))

# Show model summary
summary(nl_model_ext,
        input_data=X_day_torch,
        col_names=['input_size', 'output_size', 'num_params'])

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #
NonLinearARModel                         [6046, 100]               [6046]                    --
├─Flatten: 1-1                           [6046, 100]               [6046, 100]               --
├─Sequential: 1-2                        [6046, 100]               [6046, 1]                 --
│    └─Linear: 2-1                       [6046, 100]               [6046, 50]                5,050
│    └─ReLU: 2-2                         [6046, 50]                [6046, 50]                --
│    └─Dropout: 2-3                      [6046, 50]                [6046, 50]                --
│    └─Linear: 2-4                       [6046, 50]                [6046, 1]                 51
Total params: 5,101
Trainable params: 5,101
Non-trainable params: 0
Total mult-adds (M): 30.84
Input size (MB): 2.42
Forward/backward pass size (MB): 2.47
Params size (MB): 0.02
Estimated Total Size (MB): 4.91

In [ ]:
# Train the nonlinear AR model with extended features
nl_optimizer_ext = RMSprop(nl_model_ext.parameters(), lr=0.001)
nl_module_ext = SimpleModule.regression(nl_model_ext,
                                        optimizer=nl_optimizer_ext,
                                        metrics={'r2': R2Score()})

# Create a trainer
nl_trainer_ext = Trainer(deterministic=True,
                         max_epochs=100,
                         callbacks=[ErrorTracker()])

# Fit the nonlinear AR model to the data
nl_trainer_ext.fit(nl_module_ext, datamodule=nyse_nl_dm_ext)
nl_trainer_ext.test(nl_module_ext, datamodule=nyse_nl_dm_ext)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type             | Params | Mode 
---------------------------------------------------
0 | model | NonLinearARModel | 3.3 K  | train
1 | loss  | MSELoss          | 0      | train
---------------------------------------------------
3.3 K     Trainable params
0         Non-trainable params
3.3 K     Total params
0.013     Total estimated model params size (MB)
8         Modules in train mode
0         Modules in eval mode
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type             | Params | Mo

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=200` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss           0.5811870694160461
         test_r2            0.4484251141548157
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.5811870694160461, 'test_r2': 0.4484251141548157}]